## Corticosteroids and Tocilizumab: Overview of Clinical Trials in COVID-19

Data was retrieved from the clinicaltrial.gov API using Python. The first search was conducted under the umbrella term ”SARS-CoV-2”.Trials are filtered by “Trial type = Intervention” and “Type of intervention = Drug”. 

Corticosteroids:
A list of corticosteroid agents taken from DrugBank is used to retrieve interventional trials testing steroid drugs against covid-19 disease. Trials are filtered “randomized” in term of trial type  and “recruiting” or “completed” in terms of overall status. The final trials are viewed “manually” and some of them are selected according to the Phase of the trial and the closest primary completion date.

Tocilizumab: 


In [1]:
import pandas as pd
import requests
import numpy
import seaborn as sns
import plotly.express as px
import psutil

In [2]:
results_all = []
results = None
from_rank=0
continue_request = True
while continue_request:
    print('Requesting from {} to {}'.format(str(from_rank*100+1), str(from_rank*100+100)))
    api_url = "https://clinicaltrials.gov/api/query/study_fields?"\
    "expr=SARS-CoV-2+&fields=StudyFirstPostDate%2CNCTId%2CBriefTitle%2CCondition%2CEligibilityCriteria%2CResultsFirstPostDate%2C"\
    "Phase%2CDesignAllocation%2COverallStatus%2CInterventionName%2CStudyType%2CStartDate%2CInterventionType%2CCompletionDate&"\
    "min_rnk="+str(from_rank*100+1)+"&max_rnk="+str(from_rank*100+100)+"&fmt=json"
    if results:
        continue_request = results["StudyFieldsResponse"]["NStudiesFound"] >= from_rank*100+100
    from_rank += 1
    
    r = requests.get(api_url)
    results = r.json()
    results_all.append(results)

Requesting from 1 to 100
Requesting from 101 to 200
Requesting from 201 to 300
Requesting from 301 to 400
Requesting from 401 to 500
Requesting from 501 to 600
Requesting from 601 to 700
Requesting from 701 to 800
Requesting from 801 to 900
Requesting from 901 to 1000
Requesting from 1001 to 1100
Requesting from 1101 to 1200
Requesting from 1201 to 1300
Requesting from 1301 to 1400
Requesting from 1401 to 1500
Requesting from 1501 to 1600
Requesting from 1601 to 1700
Requesting from 1701 to 1800
Requesting from 1801 to 1900
Requesting from 1901 to 2000
Requesting from 2001 to 2100
Requesting from 2101 to 2200
Requesting from 2201 to 2300
Requesting from 2301 to 2400
Requesting from 2401 to 2500
Requesting from 2501 to 2600
Requesting from 2601 to 2700
Requesting from 2701 to 2800
Requesting from 2801 to 2900
Requesting from 2901 to 3000
Requesting from 3001 to 3100
Requesting from 3101 to 3200
Requesting from 3201 to 3300
Requesting from 3301 to 3400


In [3]:
clinical_trials_list_dfs = []
for results in results_all:
    try:
        study_fields = results["StudyFieldsResponse"]["StudyFields"]
        study_fields_new = []
        for study in study_fields:
            study_new = {}
            for key, value in study.items():
                
                if type(value) == list and key not in "Phase":
                    try:
                        study_new[key] = " ".join(value)
                    except IndexError:
                        study_new[key] = value
                else:
                    study_new[key] = value
            study_fields_new.append(study_new)
        clinical_trials_df = pd.DataFrame(study_fields_new)
        clinical_trials_list_dfs.append(clinical_trials_df)
        # display(clinical_trials_df.head())
    except KeyError:
        print("No results found")    

All trilas retreived under the search term "SARS-CoV-2" are collected as a list

In [4]:
merged_df = pd.concat(clinical_trials_list_dfs)
merged_df.head(n=10)

,BriefTitle,CompletionDate,Condition,DesignAllocation,EligibilityCriteria,InterventionName,InterventionType,NCTId,OverallStatus,Phase,Rank,ResultsFirstPostDate,StartDate,StudyFirstPostDate,StudyType
0,Association Between BMI and COVID-19,"September 30, 2020",SARS-CoV 2,,Inclusion Criteria:\n\nParticipant is aged 18 ...,Patients admitted to Intensive Care Unit with ...,Other,NCT04391738,"Active, not recruiting",[],1,,"February 1, 2020","May 18, 2020",Observational
1,"Efficacy, Safety and Immunogenicity Study of S...",September 2021,SARS-CoV2 Infection,Randomized,Inclusion Criteria:\n\nClinically healthy adul...,SARS-CoV-2 vaccine (inactivated) Placebo,Biological Biological,NCT04508075,Recruiting,[Phase 3],2,,"August 10, 2020","August 11, 2020",Interventional
2,Rapid SARS-CoV-2 IgG Antibody Testing in High ...,"January 1, 2021",SARS-CoV-2,,Inclusion Criteria:\n\nHigh risk healthcare wo...,SARS-CoV-2 IgG Antibody Testing Kit,Diagnostic Test,NCT04334876,Not yet recruiting,[],3,,"April 1, 2020","April 6, 2020",Observational
3,Antibody Detection in COVID-19 Cured Patients ...,"June 3, 2020",COVID SARS-CoV-2,,Inclusion Criteria:\n\nsigned Informed Consent...,Quantitative analysis of anti-SARS-CoV-2-antib...,Diagnostic Test Diagnostic Test,NCT04453280,Completed,[],4,,"May 18, 2020","July 1, 2020",Observational
4,Oncological Surgery in Times of COVID-19: Effe...,"December 31, 2020",SARS-CoV-2 Oncology Surgery,,Inclusion Criteria:\n\nPatients with elective ...,PT-PCR test for SARS-CoV-2,Diagnostic Test,NCT04434261,Recruiting,[],5,,"May 11, 2020","June 16, 2020",Observational
5,Humoral Immunity Against SARS-CoV-2 in Liver T...,"June 30, 2021",Liver Transplant Infection Covid19 Mortality I...,,Inclusion Criteria:\n\nPatients recovered of c...,CLIA of IgG and IgM against SARS-Cov-2,Diagnostic Test,NCT04410471,Recruiting,[],6,,"May 29, 2020","June 1, 2020",Observational
6,COVID-19 and Pregnancy Outcomes,December 2023,Coronavirus Infection Pregnancy Complications ...,,Inclusion Criteria: Pregnant women infected wi...,RT PCR SARS-CoV-2,Diagnostic Test,NCT04416373,Recruiting,[],7,,"March 22, 2020","June 4, 2020",Observational
7,Maternal-foetal Transmission of SARS-Cov-2,May 2021,Maternal Fetal Infection Transmission COVID-19...,,Inclusion Criteria:\n\nAll pregnant women SARS...,"Diagnosis of SARS-Cov2 by RT-PCR and : IgG, Ig...",Diagnostic Test,NCT04395924,Recruiting,[],8,,"May 5, 2020","May 20, 2020",Observational
8,Study of the SARS-COV-2 (COVID-19) Serological...,"July 31, 2020",Sars-CoV2,,Inclusion Criteria:\n\nArmy Training Hospital ...,Anti-SARS-CoV2 Serology Questionnaire,Diagnostic Test Other,NCT04387838,Recruiting,[],9,,"May 14, 2020","May 14, 2020",Observational
9,SARS-CoV-2 Disguise Study,"November 30, 2020",Sars-CoV2,,Inclusion Criteria:\n\nAsymptomatic medical em...,SARS-CoV-2-test SARS-CoV-2 questionnaire survey,Diagnostic Test Other,NCT04395794,Recruiting,[],10,,"May 7, 2020","May 20, 2020",Observational


The total number of trials being conducted for covid-19 disease:

In [15]:
merged_df.shape

(2811, 15)

In [16]:
merged_df['StudyFirstPostDate'] = pd.to_datetime(merged_df['StudyFirstPostDate'])

In [17]:
merged_df = merged_df.loc[merged_df['StudyFirstPostDate'] < "July 21, 2020"]

In [18]:
merged_df.shape

(2811, 15)

In [19]:
interventional_trials = merged_df.loc[merged_df["StudyType"].str.contains("Interventional")]
interventional_trials.head()

,BriefTitle,CompletionDate,Condition,DesignAllocation,EligibilityCriteria,InterventionName,InterventionType,NCTId,OverallStatus,Phase,Rank,ResultsFirstPostDate,StartDate,StudyFirstPostDate,StudyType
12,Convalescent Plasma to Limit SARS-CoV-2 Associ...,"January 31, 2023",SARS-CoV 2,Randomized,Inclusion Criteria:\n\n≥ 18 years of age\nComp...,SARS-CoV-2 convalescent plasma Plasma from a v...,Biological Biological,NCT04373460,Recruiting,[Phase 2],13,,"June 3, 2020",2020-05-04,Interventional
16,Is Adipose Tissue a Reservoir for SARS-Cov2 Sp...,"July 9, 2021",Covid-19 Patients,N/A,Inclusion Criteria:\n\nMale or female ≥ 18 yea...,biopsies of subcutaneous adipose tissue,Procedure,NCT04427878,Not yet recruiting,[Not Applicable],17,,"July 1, 2020",2020-06-11,Interventional
29,Part Two of Novel Adoptive Cellular Therapy Wi...,December 2022,COVID-19,Non-Randomized,Inclusion Criteria:\n\nAge 1 to 90 years\nTest...,SARS-CoV-2 Specific T Cells,Biological,NCT04457726,Recruiting,"[Phase 1, Phase 2]",30,,"July 1, 2020",2020-07-07,Interventional
33,Convalescent Plasma in ICU Patients With COVID...,May 2021,Covid-19 Sars-CoV2,N/A,Inclusion Criteria:\n\n18 years of age or olde...,Multiple Doses of Anti-SARS-CoV-2 convalescent...,Biological,NCT04353206,Recruiting,[Early Phase 1],34,,"June 27, 2020",2020-04-20,Interventional
34,Epidemiological Study of Seroprevalence Agains...,"July 24, 2020",Coronavirus Infection Prevalence,N/A,Inclusion Criteria:\n\nPerson who has received...,Anti-SARS-CoV-2 IgT seropositivity,Biological,NCT04448769,Completed,[Not Applicable],35,,"June 26, 2020",2020-06-26,Interventional


The number of all "Interventional" trials for covid-19:

In [20]:
interventional_trials["StudyType"].shape[0]

1595

In [21]:
drug_trials = interventional_trials.loc[interventional_trials["InterventionType"].str.contains("Drug")]
drug_trials.head()

,BriefTitle,CompletionDate,Condition,DesignAllocation,EligibilityCriteria,InterventionName,InterventionType,NCTId,OverallStatus,Phase,Rank,ResultsFirstPostDate,StartDate,StudyFirstPostDate,StudyType
42,Administration of Anti-SARS-CoV-2 Convalescent...,December 2021,COVID-19,Randomized,Inclusion Criteria:\n\nPatients ≥18 years of a...,anti-SARS-CoV-2 plasma Placebo,Drug Other,NCT04467151,Not yet recruiting,[Phase 2],43,,August 2020,2020-07-10,Interventional
55,Feasibility Study of Anti-SARS-CoV-2 Plasma Tr...,"December 31, 2020",Severe Acute Respiratory Syndrome COVID,N/A,Inclusion Criteria:\n\nLaboratory confirmed CO...,SARS-CoV-2 plasma,Drug,NCT04411602,Recruiting,[Phase 1],56,,"April 7, 2020",2020-06-02,Interventional
70,Hydroxychloroquine Chemoprophylaxis for COVID-...,"October 15, 2020",SARS-CoV-2 Healthcare Workers,Randomized,Inclusion Criteria:\n\nAdult healthcare worker...,Hydroxychloroquine Placebo oral tablet,Drug Drug,NCT04370015,Not yet recruiting,[Not Applicable],71,,"May 15, 2020",2020-04-30,Interventional
74,Prevention of SARS-CoV-2 in Hospital Workers s...,"July 15, 2020",SARS-CoV-2 Infection,Randomized,Inclusion Criteria:\n\nHospital workers workin...,hydroxychloroquine azithromycin hydroxychloroq...,Drug Drug Drug,NCT04344379,Suspended,[Phase 3],75,,"April 17, 2020",2020-04-14,Interventional
83,Peginterferon Lambda-1a for the Prevention and...,December 2021,Sars-CoV2,Randomized,Inclusion Criteria:\n\nWilling and able to pro...,Peginterferon lambda alfa-1a subcutaneous inje...,Drug Other,NCT04344600,Recruiting,[Phase 2],84,,"May 29, 2020",2020-04-14,Interventional


The number of interventional trials with "intervention type = Drug" for covid-19:

In [26]:
drug_trials.shape

(865, 16)

In [27]:
drug_trials["InterventionName_lower"]=drug_trials["InterventionName"].str.lower()


c:\users\katerina\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


Ret

In [28]:
drug_trials_tocilizumab = drug_trials.loc[drug_trials["InterventionName_lower"].str.contains("tocilizumab")]

35 trials containing tocilizumab found.

In [17]:
drug_trials_tocilizumab.shape

(35, 16)

In [26]:
drug_trials_tocilizumab

,BriefTitle,CompletionDate,Condition,DesignAllocation,EligibilityCriteria,InterventionName,InterventionType,NCTId,OverallStatus,Phase,Rank,ResultsFirstPostDate,StartDate,StudyFirstPostDate,StudyType,InterventionName_lower
30,Efficacy of Tocilizumab on Patients With COVID-19,"October 1, 2020",SARS-CoV 2,Randomized,Inclusion criteria:\n\nSubjects who meet all o...,Tocilizumab Placebos,Drug Drug,NCT04356937,"Active, not recruiting",[Phase 3],731,,"April 20, 2020",2020-04-22,Interventional,tocilizumab placebos
50,Prospective Study in Patients With Advanced or...,August 2020,SARS-CoV-2 (COVID-19) Infection Advanced or Me...,Randomized,Inclusion Criteria:\n\nI1. Age 18 or older at ...,Chloroquine analog (GNS651) Nivolumab Tocilizu...,Drug Drug Drug Other Drug Drug,NCT04333914,Suspended,[Phase 2],751,,"April 15, 2020",2020-04-03,Interventional,chloroquine analog (gns651) nivolumab tocilizu...
22,Clinical Trial of Combined Use of Hydroxychlor...,October 2020,COVID-19,Randomized,Inclusion Criteria:\n\nSubject (or authorized ...,Tocilizumab Hydroxychloroquine Azithromycin,Drug Drug Drug,NCT04332094,Recruiting,[Phase 2],1023,,"April 2, 2020",2020-04-02,Interventional,tocilizumab hydroxychloroquine azithromycin
40,Tocilizumab Treatment in Patients With COVID-19,"August 1, 2021",Sars-CoV2,N/A,Inclusion Criteria:\n\nPatients 18 years or ol...,Tocilizumab,Drug,NCT04363853,Recruiting,[Phase 2],1041,,"June 1, 2020",2020-04-27,Interventional,tocilizumab
66,Personalised Immunotherapy for SARS-CoV-2 (COV...,"April 1, 2022",COVID-19 Virus Diseases Macrophage Activation ...,Non-Randomized,Inclusion Criteria:\n\nAge equal to or above 1...,Anakinra Tocilizumab,Drug Drug,NCT04339712,Recruiting,[Phase 2],1067,,"April 2, 2020",2020-04-09,Interventional,anakinra tocilizumab
7,Study to Evaluate the Efficacy and Safety of T...,"October 31, 2020",COVID-19,Randomized,Inclusion Criteria:\n\n1) Hospitalised symptom...,Tocilizumab Methylprednisolone,Drug Drug,NCT04345445,Not yet recruiting,[Phase 3],1208,,"April 15, 2020",2020-04-14,Interventional,tocilizumab methylprednisolone
32,Tocilizumab in the Treatment of Coronavirus In...,October 2020,SARS-CoV-2 Infection,Randomized,Inclusion Criteria:\n\nI (first step):\n\nAdmi...,Tocilizumab (TCZ) Placebo,Drug Drug,NCT04335071,Recruiting,[Phase 2],1233,,"April 26, 2020",2020-04-06,Interventional,tocilizumab (tcz) placebo
76,Ultra Low Doses of Therapy With Radiation Appl...,"April 21, 2021","Pneumonia, Viral Cytokine Storm",N/A,Inclusion Criteria:\n\nAge > 18 years-old.\nDi...,Ultra-Low-dose radiotherapy ventilatory suppor...,Radiation Device Drug Drug Drug Drug Drug Drug...,NCT04394182,Recruiting,[Not Applicable],1277,,"April 21, 2020",2020-05-19,Interventional,ultra-low-dose radiotherapy ventilatory suppor...
2,Efficacy of Early Administration of Tocilizuma...,"June 6, 2020",COVID-19 Pneumonia,Randomized,Inclusion Criteria:\n\nage > 18 years\nInforme...,Tocilizumab,Drug,NCT04346355,Terminated,[Phase 2],1303,,"March 31, 2020",2020-04-15,Interventional,tocilizumab
45,The Use of Tocilizumab in the Management of Pa...,"May 8, 2021",Covid19 Pneumonia,Randomized,Inclusion Criteria:\n\nAny gender\nAge 18 and ...,Tocilizumab,Drug,NCT04377750,Recruiting,[Phase 4],1446,,"April 8, 2020",2020-05-06,Interventional,tocilizumab


In [27]:
sns.set(style="darkgrid")
# ax = sns.countplot(x="Phase", hue="OverallStatus", data=drug_trials)

In [28]:
corticosteroids = ['dexamethasone', 'budesonide', 'prednisone', 
                   "prednisolone", "methylprednisolone", "hydrocortisone", 
                   "fluticasone", "fluticasone propionate", "bethamethasone",
                   "mometasone", "mometasone furoate",
                   "beclomethasone dipropionate",
                  "flunisolide", "steroid", "corticosteroid", "glucocorticosteroid",]

In [29]:
steroids_list = []

drug_trials["BriefTitle_lower"] = drug_trials["BriefTitle"].str.lower()
for drug in corticosteroids:
    steroid_df = drug_trials.loc[
        (drug_trials["InterventionName_lower"].str.contains(drug)) |
        (drug_trials["BriefTitle_lower"].str.contains(drug))
    ]
    steroids_list.append(steroid_df)

merged_steroids = pd.concat(steroids_list)

c:\users\katerina\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


KeyError: 'InterventionName_lower'

In [30]:
merged_steroids.drop_duplicates(subset ="Rank", inplace = True)

NameError: name 'merged_steroids' is not defined

In [24]:
merged_steroids.shape

NameError: name 'merged_steroids' is not defined

In [25]:
merged_steroids = merged_steroids.loc[merged_steroids["OverallStatus"]!="Not yet recruiting"]

NameError: name 'merged_steroids' is not defined

In [25]:
merged_steroids.head()

,BriefTitle,CompletionDate,Condition,DesignAllocation,EligibilityCriteria,InterventionName,InterventionType,NCTId,OverallStatus,Phase,Rank,ResultsFirstPostDate,StartDate,StudyFirstPostDate,StudyType,InterventionName_lower,BriefTitle_lower
68,Efficacy of Dexamethasone Treatment for Patien...,"October 30, 2020",Acute Respiratory Distress Syndrome Caused by ...,Randomized,Inclusion Criteria:\n\nage 18 years or older;\...,Dexamethasone,Drug,NCT04325061,Recruiting,[Phase 4],969,,"April 3, 2020",2020-03-27,Interventional,dexamethasone,efficacy of dexamethasone treatment for patien...
4,Dexamethasone and Oxygen Support Strategies in...,"December 31, 2020",Acute Hypoxemic Respiratory Failure COVID-19,Randomized,Inclusion Criteria:\n\nAge ≥ 18 years\nAdmitte...,Dexamethasone injection placebo conventional o...,Drug Drug Procedure Procedure Procedure Procedure,NCT04344730,Recruiting,[Not Applicable],1805,,"April 10, 2020",2020-04-14,Interventional,dexamethasone injection placebo conventional o...,dexamethasone and oxygen support strategies in...
3,"NA-831, Atazanavir and Dexamethasone Combinati...","February 15, 2021",Coronavirus Infection Severe Acute Respiratory...,Randomized,Inclusion Criteria:\n\nHospitalization for man...,Drug: NA-831 NA-831 and Atazanavir NA-831and D...,Drug Combination Product Combination Product C...,NCT04452565,Recruiting,"[Phase 2, Phase 3]",1904,,"August 15, 2020",2020-06-30,Interventional,drug: na-831 na-831 and atazanavir na-831and d...,"na-831, atazanavir and dexamethasone combinati..."
32,COVID-19-associated ARDS Treated With Dexameth...,"July 22, 2020","Coronavirus Infection Pneumonia, Viral Acute R...",Randomized,Inclusion Criteria:\n\nProbable or confirmed i...,Dexamethasone,Drug,NCT04327401,Terminated,[Phase 3],2033,,"April 13, 2020",2020-03-31,Interventional,dexamethasone,covid-19-associated ards treated with dexameth...
93,Dexamethasone for COVID-19 Related ARDS: a Mul...,"January 31, 2021","Respiratory Distress Syndrome, Adult Covid-19",Randomized,Inclusion Criteria:\n\nARDS according to Berli...,Dexamethasone (high dose),Drug,NCT04395105,Recruiting,[Phase 3],2194,,"May 21, 2020",2020-05-20,Interventional,dexamethasone (high dose),dexamethasone for covid-19 related ards: a mul...


In [40]:
merged_steroids_randomized = merged_steroids.loc[merged_steroids["DesignAllocation"] == "Randomized"]

In [41]:
merged_steroids_randomized.head()

,BriefTitle,CompletionDate,Condition,DesignAllocation,EligibilityCriteria,InterventionName,InterventionType,NCTId,OverallStatus,Phase,Rank,ResultsFirstPostDate,StartDate,StudyFirstPostDate,StudyType,InterventionName_lower,BriefTitle_lower
68,Efficacy of Dexamethasone Treatment for Patien...,"October 30, 2020",Acute Respiratory Distress Syndrome Caused by ...,Randomized,Inclusion Criteria:\n\nage 18 years or older;\...,Dexamethasone,Drug,NCT04325061,Recruiting,[Phase 4],969,,"April 3, 2020",2020-03-27,Interventional,dexamethasone,efficacy of dexamethasone treatment for patien...
4,Dexamethasone and Oxygen Support Strategies in...,"December 31, 2020",Acute Hypoxemic Respiratory Failure COVID-19,Randomized,Inclusion Criteria:\n\nAge ≥ 18 years\nAdmitte...,Dexamethasone injection placebo conventional o...,Drug Drug Procedure Procedure Procedure Procedure,NCT04344730,Recruiting,[Not Applicable],1805,,"April 10, 2020",2020-04-14,Interventional,dexamethasone injection placebo conventional o...,dexamethasone and oxygen support strategies in...
3,"NA-831, Atazanavir and Dexamethasone Combinati...","February 15, 2021",Coronavirus Infection Severe Acute Respiratory...,Randomized,Inclusion Criteria:\n\nHospitalization for man...,Drug: NA-831 NA-831 and Atazanavir NA-831and D...,Drug Combination Product Combination Product C...,NCT04452565,Recruiting,"[Phase 2, Phase 3]",1904,,"August 15, 2020",2020-06-30,Interventional,drug: na-831 na-831 and atazanavir na-831and d...,"na-831, atazanavir and dexamethasone combinati..."
32,COVID-19-associated ARDS Treated With Dexameth...,"July 22, 2020","Coronavirus Infection Pneumonia, Viral Acute R...",Randomized,Inclusion Criteria:\n\nProbable or confirmed i...,Dexamethasone,Drug,NCT04327401,Terminated,[Phase 3],2033,,"April 13, 2020",2020-03-31,Interventional,dexamethasone,covid-19-associated ards treated with dexameth...
93,Dexamethasone for COVID-19 Related ARDS: a Mul...,"January 31, 2021","Respiratory Distress Syndrome, Adult Covid-19",Randomized,Inclusion Criteria:\n\nARDS according to Berli...,Dexamethasone (high dose),Drug,NCT04395105,Recruiting,[Phase 3],2194,,"May 21, 2020",2020-05-20,Interventional,dexamethasone (high dose),dexamethasone for covid-19 related ards: a mul...


In [42]:
merged_steroids_randomized.shape

(27, 17)

Illustrating selected trials according to start and ending date

In [48]:
final_steroids_df = pd.read_excel("steroids_final_data.xlsx")

In [49]:
final_steroids_df

,NCTId,Experimental,Combination treatment,Experimental or Control,Condition,Phase,Actual Start Date,Primary Completion Date,Sponsor
0,NCT04320615,Tocilizumab,-,Placebo,Covid-19 Pneumonia,III,2020-04-03,2020-07-28,Hoffmann-La Roche
1,NCT04381936,Dexamethasone,-,"Antivirals, antibiotics, anti-inflammatory agents",SARS-CoV-2 infection,II/III,2020-03-19,2021-12-31,University of Oxford
2,NCT04325061,Dexamethasone,-,Standard intensive care,ARDS caused by covid-19,IV,2020-04-03,2020-10-30,Dr. Negrin University Hospital
3,NCT04416399,Budesonide,-,Standard care,Early covid-19 Infection,II,2020-07-16,2020-09-15,University of Oxford
4,NCT04331054,Budesonide,Formoterol,Standard care,Covid-19 infection,III,2020-04-13,2020-07-13,Hôpitaux de Paris
5,NCT04451174,Prednisone,-,Standard care,Moderate covid-19 Pneumonia,III,2020-06-23,2020-09-30,University of Chile
6,NCT04263402,Methylprednisolone,-,Methylprednisolone higher dose,Covid-19 severe pneumonia,IV,2020-02-01,2020-06-01,Tongji Hospital
7,NCT04485429,Methylprednisolone,Heparine,Heparine,Covid-19 pneumonia,III,2020-07-20,2020-12-31,D'Or Institute for Research and Education
8,NCT04273321,Methylprednisolone,-,-,Covid-19 pneumonia,-,2020-02-14,2020-04-15,Beijing Chao Yang Hospital
9,NCT04349410,Methylprednisolone,-,"Antibiotics, Tocilizumab, Remdesivir",Covid-19,II/III,2020-04-11,2020-10-11,The Camelot Foundation


In [50]:
fig = px.timeline(final_steroids_df, x_start="Actual Start Date", x_end="Primary Completion Date", y="Sponsor", color="Experimental")
#fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()